In [1]:
import pandas as pd
import numpy as np

In [7]:
train = pd.read_csv("data/train_combine.csv")
train.head()

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,5,NaN,NaN,17.24,3648.0,-0.0711,NaN,1.9,696.0,3.6


In [20]:
y = train['x_e_out [-]']
X = train.drop(columns=['id','x_e_out [-]'])
X_copy = X

In [18]:
# def preprocess_data(df):
#     df['chf_exp [MW/m2]'] = np.log(df['chf_exp [MW/m2]'])
#     df['D_e [mm]'] = np.log(df['D_e [mm]'])
#     df['D_h [mm]'] = np.log(df['D_h [mm]'])
#     df['length [mm]'] = np.log(df['length [mm]'])
#     return df

# preprocess_data(X_copy)
# X_copy.head()

,author,geometry,pressure [MPa],mass_flux [kg/m2-s],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,Thompson,tube,7.00,3770.0,NaN,2.379546,6.068426,1.280934
1,Thompson,tube,NaN,6049.0,2.332144,2.332144,6.635947,1.824549
2,Thompson,NaN,13.79,2034.0,2.041220,2.041220,6.124683,0.916291
3,Beus,annulus,13.79,3679.0,1.722767,2.721295,7.665753,1.098612
4,NaN,NaN,17.24,3648.0,NaN,0.641854,6.545350,1.280934


In [21]:
X.head()

,author,geometry,pressure [MPa],mass_flux [kg/m2-s],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,Thompson,tube,7.00,3770.0,NaN,10.8,432.0,3.6
1,Thompson,tube,NaN,6049.0,10.3,10.3,762.0,6.2
2,Thompson,NaN,13.79,2034.0,7.7,7.7,457.0,2.5
3,Beus,annulus,13.79,3679.0,5.6,15.2,2134.0,3.0
4,NaN,NaN,17.24,3648.0,NaN,1.9,696.0,3.6


In [22]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline

In [23]:
# set up preprocessing for numeric columns
imp_knn = KNNImputer(n_neighbors=20)
scaler = StandardScaler()

In [24]:
# set up preprocessing for categorical columns
from sklearn.impute import SimpleImputer
imp_constant = SimpleImputer(strategy='constant')
ohe = OneHotEncoder(handle_unknown='ignore')

In [25]:
from sklearn.compose import make_column_selector, make_column_transformer

In [26]:
# select columns by data type
num_cols = make_column_selector(dtype_include='number')
cat_cols = make_column_selector(dtype_exclude='number')

In [27]:
preprocessor = make_column_transformer(
    (make_pipeline(imp_knn, scaler), num_cols),
    (make_pipeline(imp_constant, ohe), cat_cols))

In [42]:
import lightgbm as lgb
lgbm = lgb.LGBMRegressor(n_estimators=125,random_state = 318) #0.0054248

from catboost import CatBoostRegressor
cb = CatBoostRegressor(n_estimators=1000,random_state = 318,verbose=0, objective='RMSE') #0.005509

from sklearn.ensemble import GradientBoostingRegressor,HistGradientBoostingRegressor
hgbr = HistGradientBoostingRegressor(max_iter=1000,random_state=318)
gbr = GradientBoostingRegressor(n_estimators=750)

import xgboost as xgb
xgb = xgb.XGBRegressor(n_estimators=35,random_state = 318) #0.005579

from sklearn.ensemble import VotingRegressor
vr1 = VotingRegressor([('hgbr',hgbr),('lgbm',lgbm),('cb',cb)])
vr2 = VotingRegressor([('hgbr',hgbr),('lgbm',lgbm),('cb',cb),('xgb',xgb)])
vr3 = VotingRegressor([('hgbr',hgbr),('lgbm',lgbm),('cb',cb),('xgb',xgb),('gbr',gbr)])

In [29]:
pipe_lgbm = make_pipeline(preprocessor, lgbm)

In [30]:
pipe_lgbm.steps

[('columntransformer',
  ColumnTransformer(transformers=[('pipeline-1',
                                   Pipeline(steps=[('knnimputer',
                                                    KNNImputer(n_neighbors=20)),
                                                   ('standardscaler',
                                                    StandardScaler())]),
                                   <sklearn.compose._column_transformer.make_column_selector object at 0x000001632CA33D90>),
                                  ('pipeline-2',
                                   Pipeline(steps=[('simpleimputer',
                                                    SimpleImputer(strategy='constant')),
                                                   ('onehotencoder',
                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                   <sklearn.compose._column_transformer.make_column_selector object at 0x000001632CBB7FA0>)])),
 ('lgbmregress

In [31]:
#baseline of lgbm
from sklearn.model_selection import cross_val_score
cross_val_score(pipe_lgbm, X, y, scoring='neg_mean_squared_error').mean()

-0.005424763250453465

In [32]:
pipe_cb = make_pipeline(preprocessor, cb)

In [33]:
cross_val_score(pipe_cb, X, y, scoring='neg_mean_squared_error').mean()

-0.005484776429228461

In [34]:
pipe_gbr = make_pipeline(preprocessor, gbr)

In [35]:
cross_val_score(pipe_gbr, X, y, scoring='neg_mean_squared_error').mean()

-0.005600213750946591

In [38]:
pipe_hgbr = make_pipeline(preprocessor, hgbr)

In [39]:
cross_val_score(pipe_hgbr, X, y, scoring='neg_mean_squared_error').mean()

-0.005462047756620175

In [40]:
pipe_xgb = make_pipeline(preprocessor, xgb)

In [41]:
cross_val_score(pipe_xgb, X, y, scoring='neg_mean_squared_error').mean()

-0.005563941320603681

In [43]:
pipe_vr1 = make_pipeline(preprocessor, vr1)

In [44]:
cross_val_score(pipe_vr1, X, y, scoring='neg_mean_squared_error').mean()

-0.005403493046271321

In [45]:
pipe_vr2 = make_pipeline(preprocessor, vr2)

In [46]:
cross_val_score(pipe_vr2, X, y, scoring='neg_mean_squared_error').mean()

-0.005408567515407955

In [47]:
pipe_vr3 = make_pipeline(preprocessor, vr3)

In [48]:
cross_val_score(pipe_vr3, X, y, scoring='neg_mean_squared_error').mean()

-0.0054142359458214055

In [49]:
from sklearn.model_selection import GridSearchCV

params1 = {}
params1['votingregressor__weights'] = [(1,2,1),(2,1,1),(1,1,2)]

grid1 = GridSearchCV(pipe_vr1, params1, scoring='neg_mean_squared_error')
%time grid1.fit(X,y)

Wall time: 4min 37s


GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('knnimputer',
                                                                                          KNNImputer(n_neighbors=20)),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001632CA33D90>),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                     

In [51]:
grid1.best_params_

{'votingregressor__weights': (1, 2, 1)}

In [52]:
grid1.best_score_

-0.0054014158855837605

- best one

In [50]:
from sklearn.model_selection import GridSearchCV

params2 = {}
params2['votingregressor__weights'] = [(1,1,2,1),(2,1,1,1),(1,2,1,1)]

grid2 = GridSearchCV(pipe_vr2, params2, scoring='neg_mean_squared_error')
%time grid2.fit(X,y)

Wall time: 4min 42s


GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('knnimputer',
                                                                                          KNNImputer(n_neighbors=20)),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001632CA33D90>),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                     

In [53]:
grid2.best_params_

{'votingregressor__weights': (1, 2, 1, 1)}

In [55]:
grid2.best_score_

-0.005403721398422356

In [56]:
#处理test值
test = pd.read_csv('data/test.csv')
X_test = test.drop(columns=['id','x_e_out [-]'])

In [57]:
result = grid1.predict(X_test)

In [58]:
result

array([-0.01228115, -0.09907373, -0.04192901, ...,  0.02717357,
       -0.02206083, -0.01444748])

In [59]:
pd.DataFrame({'id':test.id, 'x_e_out [-]':result}).set_index('id').to_csv('sub_combine_knn_impute_vr.csv')

In [ ]:
- 0.75136